# Running the model locally

The following line will move the notebook to the root directory of the project

In [1]:
cd ../src

/home/santiagxf/repos/devsquad/trunkbased-mlops/src


In [2]:
import mlflow
from jobtools.arguments import file2namespace
from hatedetection.train.trainer import train_and_evaluate

In [3]:
params = file2namespace('../jobs/hatedetection/train.params.yml')

In [5]:
train_and_evaluate(input_dataset = '../datasets/portuguese-hate-speech-tweets/data/2019-05-28_portuguese_hate_speech_hierarchical_classification.csv',
                   eval_dataset = '../datasets/portuguese-hate-speech-tweets-eval/data/*.csv',
                   params = params)

Some weights of the model checkpoint at neuralmind/bert-base-portuguese-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


RuntimeError: CUDA out of memory. Tried to allocate 30.00 MiB (GPU 0; 6.00 GiB total capacity; 4.08 GiB already allocated; 0 bytes free; 4.33 GiB reserved in total by PyTorch)

In [8]:
import torch
print(torch.cuda.memory_summary())

|===========================================================================|
|                  PyTorch CUDA memory summary, device ID 0                 |
|---------------------------------------------------------------------------|
|            CUDA OOMs: 1            |        cudaMalloc retries: 1         |
|===========================================================================|
|        Metric         | Cur Usage  | Peak Usage | Tot Alloc  | Tot Freed  |
|---------------------------------------------------------------------------|
| Allocated memory      |    4177 MB |    4177 MB |    6086 MB |    1909 MB |
|       from large pool |    4175 MB |    4175 MB |    6084 MB |    1909 MB |
|       from small pool |       1 MB |       1 MB |       1 MB |       0 MB |
|---------------------------------------------------------------------------|
| Active memory         |    4177 MB |    4177 MB |    6086 MB |    1909 MB |
|       from large pool |    4175 MB |    4175 MB |    6084 MB |

## Authenticating against Azure

In [3]:
from azureml.core.workspace import Workspace
from azureml.core.authentication import InteractiveLoginAuthentication

In [4]:
interactive_auth = InteractiveLoginAuthentication(tenant_id="b2fbd4a8-a281-4bb1-b839-f18277513787")

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


In [5]:
ws = Workspace.from_config(path='../workspaces/dev/workspace.json', auth=interactive_auth)

## Running the model

In [6]:
import hatedetection.score.score_transformer as scorer

We re indicating `from_workspace` to indicate that this model is running locally, and `ws` indicates the workspace from where the model will be pulled from:

In [7]:
scorer.init(from_workspace=True, workspace=ws)

> Note: This may take a while since the model is quite big (1.24 GB)

Loading some data:

In [8]:
import pandas as pd

In [13]:
df = pd.read_csv('../datasets/relatus-hate-speech-text/data/speechs-sample.csv').drop(columns=['hate']).loc[:5]

Running the model:

In [14]:
scorer.run(raw_data = df)

[0.6059866547584534,
 0.30062082409858704,
 0.4073381721973419,
 0.6878570318222046,
 0.2586149275302887,
 0.6854346394538879]

In [2]:
from hatedetection.model.evaluation import compare
from common.models.model_management import get_model
import azureml.core as aml

In [3]:
eval_dataset = '../datasets/portuguese-hate-speech-tweets-eval/data/2019-05-28_portuguese_hate_speech_hierarchical_classification.csv'
confidence = 0.05
model_name = 'hate-pt-speech'

In [4]:
ws = aml.Workspace.from_config('../workspaces/dev/workspace.json')

If you run your code in unattended mode, i.e., where you can't give a user input, then we recommend to use ServicePrincipalAuthentication or MsiAuthentication.
Please refer to aka.ms/aml-notebook-auth for different authentication mechanisms in azureml-sdk.


In [5]:
champion_model = get_model(ws, 'hate-pt-speech', version="stage=production")

In [6]:
challenger_model = get_model(ws, 'hate-pt-speech', version="latest")

In [7]:
champion_path = champion_model.download("champion", exist_ok=True)

In [8]:
challenger_path = challenger_model.download("challenger", exist_ok = True)

In [9]:
compare(champion_path, challenger_path, eval_dataset, confidence)

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


RuntimeError: Found no NVIDIA driver on your system. Please check that you have an NVIDIA GPU and installed a driver from http://www.nvidia.com/Download/index.aspx

In [27]:
from importlib import reload
import hatedetection.model.hate_detection_classifier

In [28]:
reload(hatedetection.model.hate_detection_classifier)

<module 'hatedetection.model.hate_detection_classifier' from '/home/santiagxf/repos/devsquad/trunkbased-mlops/src/hatedetection/model/hate_detection_classifier.py'>